# 7.12 Orifice Size for Given Pressure Drop and Velocity

A 12" schedule 40 steel pipe is 60 feet long; it serves
to connect water from a reservoir to the atmosphere.
The water is 60 deg F and the elevation distance of the pipe
is 12 feet.

The pipe also contains a gate valve 10' from the entrance. The entrance into the reservoir is at a distance from the wall.

Find the diameter of a standard orifice plate to make the flow velocity exactly 10 ft/s.


In [1]:
from fluids.units import *
from fluids.constants import g
from math import pi
from scipy.optimize import newton
g = g*u.m/u.s**2

dH = 12*u.foot
L = (60)*u.foot
mu = 1.1*u.cP
rho = 62.364*u.lb/u.ft**3

NPS, Di, Do, t = nearest_pipe(Do=12*u.inch, schedule='40')

v = 10*u.foot/u.s
A = 0.25*pi*Di**2
m = v*A*rho
Re = rho*v*Di/mu

# Approximately match their friction factor, roughness not specified
fd = friction_factor(Re=Re, eD=0.0006*u.inch/Di)
ft = ft_Crane(Di) # Not needed

K_entrance = entrance_distance(Di=Di, method='Crane')
K_exit = exit_normal()
K_gate = K_gate_valve_Crane(D1=Di, D2=Di, angle=0.0*u.degrees)

K_tot = K_entrance + K_exit + K_gate
K_tot += K_from_f(fd=fd, L=L, D=Di)

dP = dP_from_K(K=K_tot, rho=rho, V=v)
k = 1.33

dP_gravity = rho*g*dH

dP_meter =  dP_gravity - dP
meter_type = 'ISO 5167 orifice'
orifice_taps = 'corner'

# P2 is assumed - does have an impact but it is minimal
P2 = 1e6*u.Pa
P1 = (P2 + dP_meter*2.5)
print('Numerical solver progress:')
def to_solve(beta):
    # Do not know orifice measured dP or C, only actual dP  
    D2 = Di*beta
    
    # Solve for upstream pressure given actual flow rate and guessed diameter
    P1 = differential_pressure_meter_solver(Di, rho, mu, k, D2=D2, P2=P2, 
                                       m=m, meter_type=meter_type, 
                                       taps=orifice_taps)
    
    # Calculate `C`
    C, expansibility = differential_pressure_meter_C_epsilon(Di, D2, m, P1, P2, rho, mu, k, 
                                              meter_type=meter_type, taps=orifice_taps)
    
    # Caclulate what the guessed orifice actual dP is
    calc_dP = dP_orifice(D=Di, Do=D2, P1=P1, P2=P2, C=C)
    err = (calc_dP - dP_meter).to_base_units()
    print(err.magnitude, beta, C.magnitude)
    return err.magnitude
beta = newton(to_solve, .5, tol=1e-13)
print('Size is:')
print((Di*beta).to(u.inch))

Numerical solver progress:
131456.15219585435 0.5 0.6039115256189872
131207.12176475654 0.50015 0.6039153509057876
45473.862634446166 0.5791807761904849 0.6054825449702658
22370.32441146282 0.6210574900828658 0.6055902955169561
7540.117114796503 0.6616052047236758 0.6047579998102253
1943.8952815589291 0.6822208651128452 0.603821274023946
240.43355375056126 0.689381890202793 0.6033941586008842
9.046411468669248 0.6903926265655572 0.6033292369143312
0.0442207298365247 0.6904321427501375 0.6033266747856294
8.184524631360546e-06 0.6904323368622791 0.6033266621954346
4.729372449219227e-11 0.6904323368982127 0.6033266621931039
Size is:
8.241152594519694 inch


The answer given in Crane is 7.94 inches, but only three iterations are performed there.